In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [3]:

%%time
from pandas import DataFrame

from storage import Storage
s = Storage()

from ha_utils import HeaderAnalysis
ha = HeaderAnalysis(verbose=False)

from scrape_utils import WebScrapingUtilities
wsu = WebScrapingUtilities()
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']

from cypher_utils import CypherUtilities
cu = CypherUtilities(uri=uri, user=user, password=password, driver=None, s=s, ha=ha)

from hc_utils import HeaderCategories
hc = HeaderCategories(cu=cu, verbose=False)

from section_utils import SectionUtilities
su = SectionUtilities(s=s, ha=ha, cu=cu, verbose=False)

from lr_utils import LrUtilities
lru = LrUtilities(ha=ha, cu=cu, hc=hc, verbose=False)
lru.build_isheader_logistic_regression_elements()
lru.build_pos_logistic_regression_elements()

import warnings
warnings.filterwarnings('ignore')

CPU times: total: 52.6 s
Wall time: 58.5 s


In [4]:

from datetime import datetime
from urllib.parse import urlparse, parse_qs
from IPython.display import clear_output
%run ../load_magic/dataframes.py


----

In [125]:

import shutil

file_path = r'C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\data\html\other_email.html'
file_name = re.sub(r'\W+', ' ',
                   'Machine Learning Scientist').strip().replace(' ', '_') + '.html'
new_file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
if not os.path.isfile(new_file_path):
    print(file_name)
    shutil.copy(file_path, os.path.join(cu.SAVES_HTML_FOLDER, file_name))
    page_soup = get_page_soup(file_path)
    div_soup = page_soup.find_all(name='div', id='jobDescriptionText')[0]
    child_strs_list = ha.get_navigable_children(div_soup, [])
    cu.ensure_filename(file_name, verbose=False)
    cu.populate_from_child_strings(child_strs_list, file_name, verbose=False)

Machine_Learning_Scientist.html


In [126]:

child_tags_list = ha.get_child_tags_list(child_strs_list)
print(child_tags_list)

['plaintext', 'strong', 'plaintext', 'p', 'plaintext', 'a', 'plaintext', 'p', 'p', 'p', 'h4', 'li', 'li', 'li', 'li', 'li', 'li', 'li', 'li', 'strong', 'plaintext', 'plaintext', 'h4', 'li', 'li', 'li', 'li', 'li', 'li', 'li', 'li', 'p', 'h4', 'li', 'li', 'li', 'li', 'li', 'strong', 'plaintext', 'h4', 'li', 'li', 'li', 'li', 'h4', 'li', 'li', 'li', 'li', 'li', 'p', 'strong', 'plaintext', 'a']


In [127]:

is_header_list = []
for is_header, child_str in zip(ha.get_is_header_list(child_strs_list), child_strs_list):
    if is_header is None:
        probs_list = lru.ISHEADER_PREDICT_PERCENT_FIT(child_str)
        idx = probs_list.index(max(probs_list))
        is_header = [True, False][idx]
    is_header_list.append(is_header)

In [128]:

feature_dict_list = hc.get_feature_dict_list(child_tags_list, is_header_list, child_strs_list)
feature_dict_list[0]

{'initial_tag': 'plaintext', 'is_header': True, 'is_task_scope': None, 'is_minimum_qualification': None, 'is_preferred_qualification': None, 'is_legal_notification': None, 'is_job_title': None, 'is_office_location': None, 'is_job_duration': None, 'is_supplemental_pay': None, 'is_educational_requirement': None, 'is_interview_procedure': None, 'is_corporate_scope': None, 'is_posting_date': None, 'is_other': None, 'child_str': 'Macro-Eyes is recruiting'}

In [129]:

feature_tuple_list = []
for feature_dict in feature_dict_list:
    feature_tuple_list.append(hc.get_feature_tuple(feature_dict, lru.pos_lr_predict_single))
feature_tuple_list[0]

('plaintext', 'Macro-Eyes is recruiting', 'O-CS')

In [130]:

from crf_utils import CrfUtilities
crf = CrfUtilities(ha=ha, hc=hc, cu=cu, verbose=False)
crf_list = crf.CRF.predict_single(crf.sent2features(feature_tuple_list))
crf_list

['O-CS', 'O-RQ', 'O-TS', 'O-TS', 'O-TS', 'O-CS', 'O-CS', 'O-TS', 'O-CS', 'O-RQ', 'O-TS', 'O-TS', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-TS', 'O-RQ', 'O-RQ', 'O-CS', 'O-TS', 'O-RQ', 'O-CS', 'O-CS', 'O-RQ', 'O-CS', 'O-TS', 'O-CS', 'O-TS', 'O-OL', 'O-CS', 'O-CS', 'O-CS', 'O-TS', 'O-SP', 'O-RQ', 'O-RQ', 'O-TS', 'O-CS', 'O-RQ', 'O-CS', 'O-TS', 'O-CS', 'O-TS', 'O-TS', 'O-O', 'O-CS', 'O-CS', 'O-SP', 'O-OL', 'O-OL', 'O-LN', 'O-CS', 'O-LN', 'O-CS']

In [131]:

db_pos_list = []
for navigable_parent in child_strs_list:
    db_pos_list = cu.append_parts_of_speech_list(navigable_parent, pos_list=db_pos_list)
db_pos_list

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

In [132]:

from matplotlib.colors import to_hex
from IPython.display import HTML, display

# Make an RGB dictionary of all the parts-of-speech symbols
rgba_dict = su.get_pos_color_dictionary()

html_str = ''
pos_list = []
for i, (crf_symbol, db_symbol) in enumerate(zip(crf_list, db_pos_list)):
    if db_symbol in [None, 'O', 'H']:
        pos_list.append(crf_symbol)
    else:
        pos_list.append(db_symbol)
print(pos_list)
indices_list = su.get_section(pos_list)
print(indices_list)
for i, (child_str, pos_symbol) in enumerate(zip(child_strs_list, pos_list)):
    rgba = rgba_dict[pos_symbol]
    hex_str = to_hex(rgba, keep_alpha=True)
    if len(indices_list) and (i == min(indices_list)):
        html_str += '<hr />'
    child_str = su.append_pos_symbol(child_str, pos_symbol, use_explanation=True)
    html_str += f'{i+0} {pos_symbol}) <span style="color:{hex_str};">{child_str}</span><br />'
    if len(indices_list) and (i == max(indices_list)):
        html_str += '<hr />'
display(HTML(html_str))

['O-CS', 'O-RQ', 'O-TS', 'O-TS', 'O-TS', 'O-CS', 'O-CS', 'O-TS', 'O-CS', 'O-RQ', 'O-TS', 'O-TS', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-TS', 'O-RQ', 'O-RQ', 'O-CS', 'O-TS', 'O-RQ', 'O-CS', 'O-CS', 'O-RQ', 'O-CS', 'O-TS', 'O-CS', 'O-TS', 'O-OL', 'O-CS', 'O-CS', 'O-CS', 'O-TS', 'O-SP', 'O-RQ', 'O-RQ', 'O-TS', 'O-CS', 'O-RQ', 'O-CS', 'O-TS', 'O-CS', 'O-TS', 'O-TS', 'O-O', 'O-CS', 'O-CS', 'O-SP', 'O-OL', 'O-OL', 'O-LN', 'O-CS', 'O-LN', 'O-CS']
[1, 9, 12, 13, 14, 15, 17, 18, 21, 24, 35, 36, 39]



----

In [207]:

# Hand-label individual child strings
idx = 45
child_str = child_strs_list[idx]
child_str = cu.clean_text(child_str)
print(child_str)

<h4>Why you want to work with us:</h4>


In [208]:

cypher_str = f'''
    MATCH (np:NavigableParents {{navigable_parent: '{child_str}'}})
    SET
        np.is_header = 'True',
        np.is_task_scope = 'False',
        np.is_minimum_qualification = 'False',
        np.is_preferred_qualification = 'False',
        np.is_legal_notification = 'False',
        np.is_job_title = 'False',
        np.is_office_location = 'False',
        np.is_job_duration = 'False',
        np.is_supplemental_pay = 'True',
        np.is_educational_requirement = 'False',
        np.is_interview_procedure = 'False',
        np.is_corporate_scope = 'True',
        np.is_posting_date = 'False',
        np.is_other = 'False'
    RETURN
        np.navigable_parent AS navigable_parent,
        np.is_header AS is_header,
        np.is_task_scope AS is_task_scope,
        np.is_minimum_qualification AS is_minimum_qualification,
        np.is_preferred_qualification AS is_preferred_qualification,
        np.is_legal_notification AS is_legal_notification,
        np.is_job_title AS is_job_title,
        np.is_office_location AS is_office_location,
        np.is_job_duration AS is_job_duration,
        np.is_supplemental_pay AS is_supplemental_pay,
        np.is_educational_requirement AS is_educational_requirement,
        np.is_interview_procedure AS is_interview_procedure,
        np.is_corporate_scope AS is_corporate_scope,
        np.is_posting_date AS is_posting_date,
        np.is_other AS is_other;'''
# print(cypher_str)
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
row_objs_list

[{'navigable_parent': '<h4>Why you want to work with us:</h4>', 'is_header': 'True', 'is_task_scope': 'False', 'is_minimum_qualification': 'False', 'is_preferred_qualification': 'False', 'is_legal_notification': 'False', 'is_job_title': 'False', 'is_office_location': 'False', 'is_job_duration': 'False', 'is_supplemental_pay': 'True', 'is_educational_requirement': 'False', 'is_interview_procedure': 'False', 'is_corporate_scope': 'True', 'is_posting_date': 'False', 'is_other': 'False'}]

In [141]:

cypher_str = f'''
    MATCH (np:NavigableParents {{navigable_parent: '{child_str}'}})
    RETURN
        np.navigable_parent AS navigable_parent,
        np.is_header AS is_header,
        np.is_task_scope AS is_task_scope,
        np.is_minimum_qualification AS is_minimum_qualification,
        np.is_preferred_qualification AS is_preferred_qualification,
        np.is_legal_notification AS is_legal_notification,
        np.is_job_title AS is_job_title,
        np.is_office_location AS is_office_location,
        np.is_job_duration AS is_job_duration,
        np.is_supplemental_pay AS is_supplemental_pay,
        np.is_educational_requirement AS is_educational_requirement,
        np.is_interview_procedure AS is_interview_procedure,
        np.is_corporate_scope AS is_corporate_scope,
        np.is_posting_date AS is_posting_date,
        np.is_other AS is_other;'''
# print(cypher_str)
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
row_objs_list

[{'navigable_parent': '<h4>Requirements</h4>', 'is_header': None, 'is_task_scope': None, 'is_minimum_qualification': None, 'is_preferred_qualification': None, 'is_legal_notification': None, 'is_job_title': None, 'is_office_location': None, 'is_job_duration': None, 'is_supplemental_pay': None, 'is_educational_requirement': None, 'is_interview_procedure': None, 'is_corporate_scope': None, 'is_posting_date': None, 'is_other': None}]

In [133]:

# Add URL
posting_url = 'https://macro-eyes.com/careers/machinelearning'
posting_url = cu.clean_text(posting_url)
cypher_str = f'''
    MATCH (fn:FileNames {{file_name: "{file_name}"}})
    SET fn.posting_url = "{posting_url}"
    RETURN fn;'''
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
    print(row_objs_list)

[{'fn': <Node id=767169 labels=frozenset({'FileNames'}) properties={'file_name': 'Machine_Learning_Scientist.html', 'posting_url': 'https://macro-eyes.com/careers/machinelearning'}>}]
